<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/logo.png?raw=true" alt="2022年度ゲノム情報解析入門" height="100px" align="middle">

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>

[課題] ゲノムワイド関連解析(GWAS)
---

　課題をはじめる前に、次のコードセルを一度実行してください。課題に必要なパッケージのインストールや、プログラム、サンプルファイルをダウンロードします。

In [ ]:
################################
##  以下の実習の前に一度実行してください。##
################################
## Rパッケージのインストール、プログラム・サンプルファイルのダウンロード
system("wget -q -O library.tar.gz https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2022/datasets/library.tar.gz")
system("wget -q -O GWAS_sample_genotype_hw.csv https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2022/datasets/GWAS_sample_genotype_hw.csv")
system("wget -q -O GWAS_sample_phenotype_hw.csv https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2022/datasets/GWAS_sample_phenotype_hw.csv")
untar("library.tar.gz")
.libPaths("library")
# 確認
library(rrBLUP)
library(ggplot2)


## データの読み込み
課題で扱うデータも、実習で扱った「ひとめぼれ系統」と「URASAN1系統」をかけ合わせたRIL集団になります。

ただし、今回扱う形質データは、３地点で計測した籾数のデータになります。

(同じ集団を青森、岩手、福島で栽培し、それぞれの地点で形質データを測定した)

In [ ]:
# 遺伝子型情報読み込み
rice_genotype <- read.csv("GWAS_sample_genotype_hw.csv")

In [ ]:
# 形質値データ読み込み&最初の数サンプルのみ表示
rice_phenotype <- read.csv("GWAS_sample_phenotype_hw.csv")
head(rice_phenotype)

## 課題1

以下の２つのコードを動かすと、岩手の籾数データでGWASを行った結果が表示されます。(実習でやったものと同じ)

その結果、籾数と関係してそうな領域(FDR<0.01)を３番染色体上に検出することが出来ます。

しかし、検出された領域には遺伝子が数百個位置しており、どの遺伝子が籾数に影響を与えるのかを決めるにはまだまだ難しい状況です。

**ここから候補領域をより狭い領域に絞っていったり、原因遺伝子を特定していきたい場合、更にどのようなアプローチを適用すると良いと考えられるでしょうか？**

自分の考えを下のAnswerの枠に記入してください。

In [ ]:
# GWAS
library(rrBLUP)
X <- rice_genotype[, 4:dim(rice_genotype)[2]]
M <- X-1
geno <- data.frame(marker=rice_genotype$Marker, chrom=rice_genotype$Chrom, pos=rice_genotype$Position, M, check.names=FALSE)
GWAS_rice <- GWAS(rice_phenotype[,c(1,2)], geno, plot=FALSE)

# calculate FDR threshold
q_values_GN <- p.adjust(10 ** (-GWAS_rice$Iwate), method = "BH")
FDR_threshold_GN <- min(GWAS_rice$Iwate[q_values_GN < 0.01])

# visualize result
g <- ggplot(GWAS_rice, aes(x = marker, y = Iwate, color=chrom))
g <- g + geom_point()
g <- g + geom_hline(yintercept = FDR_threshold_GN, linetype = "dashed")
g <- g + ggtitle("Grain number in Iwate")
g <- g + ylim(c(0, 12))
g <- g + theme(text = element_text(size = 24))
plot(g)

In [ ]:
# show candidate region
candidate <- GWAS_rice[q_values_GN < 0.01, ]
print(paste(candidate$chrom[1], min(candidate$pos), "~", max(candidate$pos), "bp が有意な領域として検出された"))

In [ ]:
#@title { vertical-output: true, display-mode: "form" }
Answer = "" #@param {type:"string"}

### **解答例**

* GWASを集団サイズを増やして再度行う
* 別集団でGWASを行った結果と重なる部分に着目する
* 遺伝子をノックアウトしていき結果を見る

などがパッと思いつく方法かと思います。

集団サイズを増やすにも、ただ適当に増やすのではなく、今回検出された候補領域(chr03 546864 ~ 1923995 bp)の遺伝子型がばらつく様な交配集団を作成するのがポイントです。

## 課題2

以下のコードを動かすと、

岩手だけではなく、３地点(青森、岩手、福島)それぞれで栽培した結果の形質データを用いたGWASの結果が表示されます。

* 青森での栽培結果を用いたGWAS
* 岩手での栽培結果を用いたGWAS(課題1と同じ)
* 福島での栽培結果を用いたGWAS

場所毎に異なったGWASの結果が得られると思いますが、

**この結果はどのように解釈することが出来るでしょうか？**

自分の考えを下のAnswerの枠に記入してください。

In [ ]:
# draw manhattan plot for Grain number
GWAS_rice_3loc <- GWAS(rice_phenotype, geno, plot=FALSE)
for (place in c("Aomori", "Iwate", "Fukushima")) {
    q_values_GN <- p.adjust(10 ** (-GWAS_rice_3loc[,place]), method = "BH")
    FDR_threshold_GN <- min(GWAS_rice_3loc[,place][q_values_GN < 0.01])

    g <- ggplot(GWAS_rice_3loc, aes_(x = as.name("marker"), y = as.name(place), color=as.name("chrom")))
    g <- g + geom_point()
    g <- g + geom_hline(yintercept = FDR_threshold_GN, linetype = "dashed")
    g <- g + ggtitle(paste("Grain number in", place))
    g <- g + ylim(c(0, 12))
    g <- g + theme(text = element_text(size = 24))
    plot(g)
}

In [ ]:
#@title { vertical-output: true, display-mode: "form" }
Answer = "" #@param {type:"string"}

### **解答例**

今回の状況は、**同じ交配集団を**3地点で栽培し、GWASを行った結果になります。(説明が足りていませんでした、すみません。)

解釈の仕方は色々とありますが、まず形質というのは遺伝子と環境の効果によって決まります。

形質値 = 遺伝子の効果 + 環境の効果

そのため、異なる場所で同じ集団を栽培した場合、以下の様な感じになります。

* 青森では、 形質値 = 遺伝子の効果(個体毎に異なる) + 青森環境の効果(全個体に共通)
* 岩手では、形質値 = 遺伝子の効果(個体毎に異なる) + 岩手環境の効果(全個体に共通)

というような感じで個体間の形質値の違いは全て遺伝子によるものと考えられます。

よって、どの場所でGWASを行ったとしても、同じ集団を使用すれば、似たような結果が出ることが想定されます。

しかし、今回はそれが違っていたということでした。これが何を表しているかというと、

形質値 = 遺伝子の効果 + 環境の効果で説明される**遺伝子の効果が場所によって異なる**ことを表しています。

これは**GxE効果(遺伝子-環境間相互作用)**と呼ばれており、特定の遺伝子では、環境条件によって遺伝子の効果が変わる、ということが報告されています。

今回の結果からは、場所が変わる(環境要因が変わる)ことで、籾数に影響を与えたり、与えなかったりする遺伝子が3番染色体上に位置している、ということが示唆されます。

# 課題の提出について

提出期限： 2022年12月12日（月） 12:00まで

提出方法：
1. ノートブックをGoogleドライブに保存する
1. 保存後、ノートブック画面右上の「共有」ボタンから共有リンクURLを作成する
  1. アクセス設定を「リンクを知っている全員」に変更する
  1. 「リンクをコピー」をクリックする
  1. 「完了」を押して共有設定ウィンドウを閉じる
1. [Panda課題ページ](https://panda.ecs.kyoto-u.ac.jp/portal/directtool/1d1d1bd9-928d-440c-a745-9b97b2a4bc04/)のフォームに共有リンクURLを貼り付けて、課題を提出する

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>